In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Ejercicio de regresión

Se quiere determinar la relación entre la concentración de cierto fármaco en el torrente sanguíneo y el tiempo transcurrido desde que se administró el fármaco. Se recopila datos sobre la concentración de la droga en el torrente sanguíneo en diferentes intervalos de tiempo después de la administración.

In [ ]:
dataset = pd.read_csv("datasets/drug.csv") # cargando los datos desde un csv
dataset.head(10) 

In [ ]:
dataset.describe()

In [ ]:
# Pandas nos da algunas herramientas de graficado
plt.figure(figsize=(7, 5))
#dataset.Time.hist()
dataset["Time"].hist()
plt.show()

In [ ]:
plt.figure(figsize=(7, 5))
dataset.Concentration.hist();

In [ ]:
# podemos ver cual es la relacion entre ambas variables....

plt.figure(figsize=(7, 5))
correlacion_drug = np.corrcoef(dataset["Time"], dataset["Concentration"])
sns.heatmap(data=correlacion_drug, annot=True, annot_kws={"size": 16})

In [ ]:
# Graficamos el dataset

plt.figure(figsize=(7, 5))
plt.scatter(dataset['Time'], dataset['Concentration'] , color='r', marker="x",s=60)
plt.grid(True, linewidth=0.5)
plt.xlabel('Tiempo [h]', fontsize=14)
plt.ylabel('Concentración [mg/L]', fontsize=14)
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)
plt.title('Dataset de concentración de droga vs. tiempo', fontsize=16)
plt.show()

Ya tenemos los datos y de hecho vemos que hay una relacion lineal entre las dos variables, por lo tanto asumimos que es podemos usar **regresion lineal simple** (ya que hay una variable independiente y otra dependiente) para poder resolver nuestro problema.

Ahora vamos a armar nuestro modelo, para ello vamos a utilizar el proceso que vimos de Machine Learning

![proceso ML](./img/proceso_ml.png)

Es decir, necesitaremos datos de entrenamiento y datos de test. Esto se puede hacer facilmente con la libreria **scikit-learn** que nos permite separar nuestros datos para entrenar y para testear el funcionamiento de nuestro modelo.

Puedes tambien visitar la pagina de scikit-learn [Aca](https://scikit-learn.org/stable/).

Para instalarlo, hacemos 

```!pip install scikit-learn```

In [ ]:
# importando modulo para separar datos de entrenamiento y testeo de scikit-learn
from sklearn.model_selection import train_test_split

In [ ]:
# Armamos array de Numpy con los features
X = dataset.iloc[:,0].values
X = X.reshape([-1, 1])
# Y con la variable dependendiente (target)
y = dataset.iloc[:,1].values

In [ ]:
# valores de X
X

In [ ]:
# valores de y
y

In [ ]:
# Cantidad de elementos en X y en y
print("Cantidad de elementos en 'X':", X.shape)
print("Cantidad de elementos en 'y':", y.shape)

Separamos el dataset de entrenamiento y testeo. Para este problema usamos el tamaño del test de 30% aprox.

In [ ]:
# Separando nuestro dataset en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Datos de train y de test
print("Valores de X_train:",X_train.size)
print("Valores de y_train:",y_train.size)
print("Valores de X_test:",len(X_test))
print("Valores de y_test:",len(y_test))

Para aplicar el modelo de regresion lineal, la forma de implementarlo usando **scikit-learn**, parametros y formas de uso la podemos encontrar [aca](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression).

In [ ]:
# Creando el modelo de regresion lineal simple:
from sklearn.linear_model import LinearRegression
regresion = LinearRegression()

In [ ]:
# Y lo entrenamos, con el set de entrenamiento
regresion.fit(X_train, y_train)

In [ ]:
# Una vez entrenado, podemos ver diferente informacion del modelo:

print(f"El valor de la interseccion de la recta sera {regresion.intercept_ }")
print(f"El valor del coeficiente de la recta sera {regresion.coef_ }")
print(f"La ecuación de la recta entonces sera la siguiente: y = {regresion.intercept_ }+({regresion.coef_[0]})X")

In [ ]:
print(f"El coeficiente de Pearson es {regresion.score(X_train, y_train)}")

In [ ]:
# Calculamos el desvío estándar del modelo
std_dev_model = np.sqrt((np.sum((y_train - regresion.predict(X_train))**2))/(y_train.size - 2))
print(f"Desvío estándar del modelo {std_dev_model}")

In [ ]:
# y si lo graficamos? graficar siempre nos dara una mejor idea de lo que sucede

plt.figure(figsize=(7, 5))
plt.title('Concentración de droga vs. tiempo', fontsize=16)
plt.xlabel('Tiempo [h]', fontsize=14)
plt.ylabel('Concentración [mg/L]', fontsize=14)
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)

plt.scatter(X_train, y_train, color='r', marker="o", s=60)
plt.plot(X_train, regresion.predict(X_train), color="b", linewidth=2)

plt.ylim(0)  # Establece el límite inferior del eje y en cero

In [ ]:
# Convertir X_train en un vector unidimensional
X_train = X_train.flatten()

# Regresión lineal
regression = np.polyfit(X_train, y_train, 1)
regression_line = np.polyval(regression, X_train)

# Calcular las distancias entre los puntos y la línea de regresión
distances = np.abs(regression_line - y_train)

# Graficar los puntos
plt.figure(figsize=(7, 5))
plt.scatter(X_train, y_train, color='r', s=20)

# Graficar la línea de regresión
plt.plot(X_train, regression_line, color='b', linewidth=2)

# Graficar las líneas perpendiculares desde cada punto a la línea de regresión
for x, y, distance in zip(X_train, y_train, distances):
    plt.plot([x, x], [y, regression[0]*x + regression[1]], color='black', linestyle='-')
    plt.text(x, y, f'{distance:.0f}', ha='left', va="baseline", fontsize=16)

# Configuraciones adicionales
plt.title("Distancia entre la línea de regresión y los puntos", fontsize=16)
plt.xlabel('Tiempo [h]', fontsize=14)
plt.ylabel('Concentración [mg/L]', fontsize=14)
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)

# Mostrar el gráfico
plt.show()

----
## Metricas

Entrenamos el modelo, pero para validar si está bien entrenado, debemos usar el dataset de testeo. 

Vamos a aplicar las siguientes metricas de evaluación usando scikit-learn:
- R2
- MAE
- MSE
- RMSE
- MAPE
- MPE

In [ ]:
#Primero obtenemos las predicciones del modelo
y_pred = regresion.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [ ]:
#sklean no tiene el error porcentual medio (MPE) lo vamos a crear nosotros
def mean_porcentual_error(yreal, ypred):

    return np.mean((yreal-ypred)/yreal)

In [ ]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
mpe = mean_porcentual_error(y_test, y_pred)
print("R-cuadrado en test:", r2)
print("Error absoluto medio:", mae)
print("Error cuadratico medio:", mse)
print("Raiz de error cuadratico medio:", rmse)
print(f"Error absoluto porcentual medio: {mape*100:.2f}%")
print(f"Error porcentual medio: {mpe*100:.2f}%")

Estas no son las unicas métricas que se pueden calcular. Scikit-learn documenta varias [metricas de regresión](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)

Una vez que tenemos el modelo, y estamos conforme, podemos guardarlo:

In [ ]:
# Podemos utilizar pickle, existen otras herramientas pero con esto bastara.
import pickle

with open('modelo_regresion_lineal.pkl', 'wb') as archivo:
    pickle.dump(regresion, archivo)

In [ ]:
#Podemos llamarlo para usarlo con otros valores y predecir segun lo que nosotros queremos.

with open('modelo_regresion_lineal.pkl', 'rb') as archivo:
    modelo_cargado = pickle.load(archivo)

In [ ]:
# Pasandole nuevos datos a por predecir.....ojo con la forma de pasarlos!
X_pred = np.array([ [1], [3.5] ]) # Quiero predecir valores para 1 y 4 horas y media

predicciones = modelo_cargado.predict(X_pred) 
predicciones

----
## Regresión lineal multiple

Aunque se uso un ejemplo de una regresión lineal simple, todo lo que vimos sirve exactamente para un problema n-dimensional.

Este dataset proviene de [acá](https://www.kaggle.com/datasets/farhanmd29/50-startups). Este conjunto de datos tiene datos recopilados de Nueva York, California y Florida sobre 50 empresas emergentes. Las variables utilizadas en el conjunto de datos son ganancias, gasto en I+D, gasto administrativo y gasto en marketing. 

Queremos predecir la ganancia usando las otras variables. Pero tenemos un problema inicial

In [ ]:
dataset = pd.read_csv("datasets/50_Startups.csv") 
dataset.head() 

In [ ]:
dataset.info()

Vemos que *State* es una variable categorica.

In [ ]:
dataset["State"].unique()

In [ ]:
dataset["State"].nunique()

In [ ]:
dataset["State"].value_counts()

Como hacemos para introducir una variable categorica en un modelo matematico? Usando variables Dummies.

In [ ]:
dataset_with_dummies = pd.get_dummies(data=dataset, columns=['State'])

In [ ]:
dataset_with_dummies.head()

In [ ]:
# Convertimos a todo en float
dataset_with_dummies = dataset_with_dummies.astype('float')
dataset_with_dummies.head()

Esto lo pueden hacer tambien con scikit-learn usando LabelEncoder, OneHotEncoder, make_column_transformer.

In [ ]:
#Quitamos una columna de las variables dummy
dataset_with_dummies.drop(columns="State_New York", inplace=True)

In [ ]:
dataset_with_dummies = dataset_with_dummies[["R&D Spend", "Administration", "Marketing Spend", "State_California", "State_Florida", "Profit"]]
dataset_with_dummies.head()

In [ ]:
# Podemos ver cual es la relacion entre variables, recordemos, es mejor con la variable objetivo, pero malo si es entre variables de entrada....
plt.figure(figsize=(7, 5))
correlacion_profit = dataset_with_dummies.corr().round(2)
sns.heatmap(data=correlacion_profit, annot=True, annot_kws={"size": 14})

In [ ]:
sns.pairplot(data=dataset_with_dummies, diag_kind="kde");

Porque tengo muchas variables no significa que debo aplicarla al modelo sin ningun criterio. Como vemos, hay variables que estan correlacionadas que nos pueden dar problemas.

Recordemos la maxima: Garbage in + garbage out.

Ademas, muchas variables es problema a futuro. Nos puede dificultar los pipelines y hacer mas dificil de entender los datos.

### Apliquemos la regresión

In [ ]:
# Armamos array de Numpy con los features
X = dataset_with_dummies.iloc[:,:-1].values
# Y con la variable dependendiente (target)
y = dataset_with_dummies.iloc[:,-1].values

In [ ]:
# Separando nuestro dataset en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Datos de train y de test
print("Dimension de X_train:",X_train.shape)
print("Valores de y_train:",y_train.size)
print("Dimension de X_test:",X_test.shape)
print("Valores de y_test:",len(y_test))

In [ ]:
regresion = LinearRegression()

regresion.fit(X_train, y_train)

print(f"El valor de la interseccion de la recta sera {regresion.intercept_ }")
print(f"Los valores de los coeficientes de la recta sera {regresion.coef_ }")

In [ ]:
print(f"El coeficiente de Pearson es {regresion.score(X_train, y_train)}")

In [ ]:
# Calculamos el desvío estándar del modelo
std_dev_model = np.sqrt((np.sum((y_train - regresion.predict(X_train))**2))/(y_train.size-6))
print(f"Desvío estándar del modelo {std_dev_model}")

In [ ]:
#Otenemos las predicciones del modelo
y_pred = regresion.predict(X_test)

In [ ]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
mpe = mean_porcentual_error(y_test, y_pred)
print("R-cuadrado en test:", r2)
print("Error absoluto medio:", mae)
print("Error cuadratico medio:", mse)
print("Raiz de error cuadratico medio:", rmse)
print(f"Error absoluto porcentual medio: {mape*100:.2f}%")
print(f"Error porcentual medio: {mpe*100:.2f}%")

----
### Selección de modelo

Para seleccionar el modelo vamos a usar otra libreria especializada en herramientas estadisticas, llamada [statsmodels](https://www.statsmodels.org/stable/index.html).

Vamos a usar la clase OLS que implementa regresiones lineales ordinarias pero ademas realiza automaticamete el calculo de criterios de selección.

In [ ]:
import statsmodels.api as sm

# El modelo de statsmodel necesita una entrada para el termino independiente.
# Para ello vamos a agregar una columna de 1 en la primera columna
X_statsmodels = np.append(arr = np.ones((X.shape[0],1)).astype(int), values = X, axis = 1)

# Nivel de significancia que vamos a usar
SL = 0.1

In [ ]:
X_statsmodels[0, :]

Vamos a aplicar el metodo de construccion de eliminación hacia atrás. Empezamos con el modelo con todas las variables. Para este proceso vamos a usar todo el conjunto de datos.

Lo que vamos a seleccionar mediante un test de hipotesis para cada atributo, si el coeficiente es $\beta_i = 0$ (hipotesis nula). Si no podemos rechazar la hipotesis nula, se elimina el coeficiente. Eliminamos de a una por vez, quitando el peor caso. Para eso ponemos un nivel de significancia de 5%, y el que sobrepase por más distancia a esto, eliminamos.

In [ ]:
X_opt = X_statsmodels[:, [0, 1, 2, 3, 4, 5]]
regression_OLS = sm.OLS(endog = y, exog = X_opt.tolist()).fit()
regression_OLS.summary()

Vemos que la variable 4 tiene el mayor nivel de significancia y supera el limite que impusimos de 0.05. La eliminamos.

In [ ]:
X_opt = X_statsmodels[:, [0, 1, 2, 3, 5]]
regression_OLS = sm.OLS(endog = y, exog = X_opt.tolist()).fit()
regression_OLS.summary()

Vemos que la variable 5 tiene el mayor nivel de significancia y supera el limite que impusimos de 0.05. La eliminamos

In [ ]:
X_opt = X_statsmodels[:, [0, 1, 2, 3]]
regression_OLS = sm.OLS(endog = y, exog = X_opt.tolist()).fit()
regression_OLS.summary()

Vemos que la variable 2 tiene el mayor nivel de significancia y supera el limite que impusimos de 0.05. La eliminamos

In [ ]:
X_opt = X_statsmodels[:, [0, 1, 3]]
regression_OLS = sm.OLS(endog = y, exog = X_opt.tolist()).fit()
regression_OLS.summary()

En este último paso, ninguna variable esta por encima del nivel de significancia. Entonces elegimos para hacer la regresión a las variables de entradas "R&D Spend" y "Marketing Spend". Recordemos que eran la que mejor correlación nos daban.

In [ ]:
# Armamos array de Numpy con los features
X = dataset_with_dummies[["R&D Spend", "Marketing Spend"]].values
# Y con la variable dependendiente (target)
y = dataset_with_dummies.iloc[:,-1].values

In [ ]:
# Separando nuestro dataset en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Datos de train y de test
print("Dimension de X_train:",X_train.shape)
print("Valores de y_train:",y_train.size)
print("Dimension de X_test:",X_test.shape)
print("Valores de y_test:",len(y_test))

In [ ]:
regresion = LinearRegression()

regresion.fit(X_train, y_train)

print(f"El valor de la interseccion de la recta sera {regresion.intercept_ }")
print(f"Los valores de los coeficientes de la recta sera {regresion.coef_ }")

In [ ]:
print(f"El coeficiente de Pearson es {regresion.score(X_train, y_train)}")

In [ ]:
# Calculamos el desvío estándar del modelo
std_dev_model = np.sqrt((np.sum((y_train - regresion.predict(X_train))**2))/(y_train.size-2))
print(f"Desvío estándar del modelo {std_dev_model}")

In [ ]:
#Otenemos las predicciones del modelo
y_pred = regresion.predict(X_test)

In [ ]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
mpe = mean_porcentual_error(y_test, y_pred)
print("R-cuadrado en test:", r2)
print("Error absoluto medio:", mae)
print("Error cuadratico medio:", mse)
print("Raiz de error cuadratico medio:", rmse)
print(f"Error absoluto porcentual medio: {mape*100:.2f}%")
print(f"Error porcentual medio: {mpe*100:.2f}%")

El modelo con el dataset de testing mejoró. Menos variable generaliza mejor.

----

## Regresión polinómica

Para este ejercicio, vamos a usar el dataset de salarios por posición. Es un pequeño ejemplo con pocos datos que nos servira solo a modo de ejemplo.

In [ ]:
dataset = pd.read_csv("datasets/position_salaries.csv")
dataset.head()

In [ ]:
plt.figure(figsize=(7, 5))
plt.scatter(dataset['Level'], dataset['Salary'] , color='g', marker="x",s=60)
plt.grid(True, linewidth=0.5)
plt.xlabel("Posición del empleado", fontsize=12)
plt.ylabel("Sueldo (en $)", fontsize=12)
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)
plt.title('Salario vs. nivel de puesto', fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(7, 5))
correlacion = dataset[["Level", "Salary"]].corr().round(2)
sns.heatmap(data=correlacion, annot=True, annot_kws={"size": 14});

La correlación es bastante alta entre nivel y salario. Un detalle importante, correlación solo mide la relación entre variables como si fuera una relación lineal. Esto nos indica que pese a como se ve el diagrama de dispersión, la componente lineal explica bastante de la relación.

Como solo tenemos muy pocos datos, vamos a separar el dataset en testing en una sola observación. Esto es a fines didacticos. 

In [ ]:
# Armamos array de Numpy con los features
X = dataset.iloc[:,1:-1].values
# Y con la variable dependendiente (target)
y = dataset.iloc[:,-1].values

In [ ]:
# Separando nuestro dataset en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1, random_state=42)

# Datos de train y de test
print("Dimension de X_train:",X_train.shape)
print("Valores de y_train:",y_train.size)
print("Dimension de X_test:",X_test.shape)
print("Valores de y_test:",len(y_test))

In [ ]:
# Empezamos con un modelo lineal
lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)

plt.figure(figsize=(7, 5))
plt.scatter(X_train, y_train, color = "red")
plt.scatter(X_test, y_test , color='g', marker="x",s=60)
plt.plot(X_train, lin_reg.predict(X_train), color = "blue")
plt.title("Modelo de Regresión Lineal", fontsize=16)
plt.xlabel("Posición del empleado", fontsize=14)
plt.ylabel("Sueldo (en $)", fontsize=14);

print(f"El coeficiente de Pearson es {lin_reg.score(X_train, y_train)}")

El módulo de scikit-learn para implementar regresiones polinómicas es [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

In [ ]:
# Sigamos con un modelo cuadrático
from sklearn.preprocessing import PolynomialFeatures
pol_reg = PolynomialFeatures(degree = 2)
# Basicamente nos crea un array con los terminos lineal y cuadraticos
X_poly = pol_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

plt.figure(figsize=(7, 5))
# Creamos valores para poder graficar "el ajuste"
X_grid = np.arange(np.min(X), np.max(X)+0.1, 0.1)
X_grid = X_grid.reshape(len(X_grid), 1)
plt.scatter(X_train, y_train, color = "red")
plt.scatter(X_test, y_test , color='g', marker="x",s=60)
plt.plot(X_grid, lin_reg_2.predict(pol_reg.fit_transform(X_grid)), color = "blue")
plt.title("Modelo de Regresión Polinomica de orden 2", fontsize=16)
plt.xlabel("Posición del empleado", fontsize=14)
plt.ylabel("Sueldo (en $)", fontsize=14);

print(f"El coeficiente de Pearson es {lin_reg_2.score(X_poly, y_train)}")

In [ ]:
# Sigamos con un modelo de orden 3
from sklearn.preprocessing import PolynomialFeatures
pol_reg = PolynomialFeatures(degree = 3)
# Basicamente nos crea un array con los terminos lineal y cuadraticos
X_poly = pol_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

plt.figure(figsize=(7, 5))
# Creamos valores para poder graficar "el ajuste"
X_grid = np.arange(np.min(X), np.max(X)+0.1, 0.1)
X_grid = X_grid.reshape(len(X_grid), 1)
plt.scatter(X_train, y_train, color = "red")
plt.scatter(X_test, y_test , color='g', marker="x",s=60)
plt.plot(X_grid, lin_reg_2.predict(pol_reg.fit_transform(X_grid)), color = "blue")
plt.title("Modelo de Regresión Polinomica de orden 3", fontsize=16)
plt.xlabel("Posición del empleado", fontsize=14)
plt.ylabel("Sueldo (en $)", fontsize=14);

print(f"El coeficiente de Pearson es {lin_reg_2.score(X_poly, y_train)}")

In [ ]:
# Sigamos con un modelo de orden 4
from sklearn.preprocessing import PolynomialFeatures
pol_reg = PolynomialFeatures(degree = 4)
# Basicamente nos crea un array con los terminos lineal y cuadraticos
X_poly = pol_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

plt.figure(figsize=(7, 5))
# Creamos valores para poder graficar "el ajuste"
X_grid = np.arange(np.min(X), np.max(X)+0.1, 0.1)
X_grid = X_grid.reshape(len(X_grid), 1)
plt.scatter(X_train, y_train, color = "red")
plt.scatter(X_test, y_test , color='g', marker="x",s=60)
plt.plot(X_grid, lin_reg_2.predict(pol_reg.fit_transform(X_grid)), color = "blue")
plt.title("Modelo de Regresión Polinomica de orden 4", fontsize=16)
plt.xlabel("Posición del empleado", fontsize=14)
plt.ylabel("Sueldo (en $)", fontsize=14);

print(f"El coeficiente de Pearson es {lin_reg_2.score(X_poly, y_train)}")

In [ ]:
# Sigamos con un modelo de orden 8
from sklearn.preprocessing import PolynomialFeatures
pol_reg = PolynomialFeatures(degree = 8)
# Basicamente nos crea un array con los terminos lineal y cuadraticos
X_poly = pol_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

plt.figure(figsize=(7, 5))
# Creamos valores para poder graficar "el ajuste"
X_grid = np.arange(np.min(X), np.max(X)+0.1, 0.1)
X_grid = X_grid.reshape(len(X_grid), 1)
plt.scatter(X_train, y_train, color = "red")
plt.scatter(X_test, y_test , color='g', marker="x",s=60)
plt.plot(X_grid, lin_reg_2.predict(pol_reg.fit_transform(X_grid)), color = "blue")
plt.title("Modelo de Regresión Polinomica de orden 8", fontsize=16)
plt.xlabel("Posición del empleado", fontsize=14)
plt.ylabel("Sueldo (en $)", fontsize=14);

print(f"El coeficiente de Pearson es {lin_reg_2.score(X_poly, y_train)}")

Vemos que el polinomio de grado 8 pasa perfectamente por todos los puntos de entrenamiento. Pero si vemos el valor de testing, el modelo predijo muy mal, inclusive predice un absurdo de sueldo negativos.

Cuando hablamos de que buscamos generalizar el modelo, hablamos de evitar estos efectos (aqui los exageramos para que sean evidentes). Esto es lo que se llama sobre-ajuste o overfitting, y significa que el modelo se entrenó para que responda muy bien a los datos de entrenamiento, pero es incapaz de predecir correctamente nuevos valores.